In [ ]:
!pip install xarray netCDF4 h5netcdf h5py requests dask --upgrade


In [ ]:
import os

# NOAA OISST v2.1 Data URL
BASE_URL = "https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr"

# Directories
BASE_DIR = os.getcwd()  # Get current working directory inside WSL
RAW_DIR = os.path.join(BASE_DIR, "content", "oisst_raw")
SUBSET_DIR = os.path.join(BASE_DIR, "content", "oisst_california_subset")
MERGED_FILE = os.path.join(BASE_DIR, "content", "oisst_california_1981_2025.nc")

# Create directories
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(SUBSET_DIR, exist_ok=True)

# Define years & months to download
YEARS = range(1981, 2026)
MONTHS = range(1, 13)

# Define California Coast region
LAT_RANGE = slice(30, 42)  # 30°N to 42°N
LON_RANGE = slice(230, 245)  # Convert -130°W to -115°W (360° system)

# Minimum valid NetCDF file size (100 KB threshold)
MIN_VALID_SIZE = 100_000


In [ ]:
print(f"RAW_DIR: {RAW_DIR}")  # Debugging output


In [ ]:
import os
import requests

for year in YEARS:
    for month in MONTHS:
        date_str = f"{year}{month:02d}01"
        month_str = f"{year}{month:02d}"

        file_url = f"{BASE_URL}/{month_str}/oisst-avhrr-v02r01.{date_str}.nc"
        output_file = os.path.join(RAW_DIR, f"oisst-avhrr-v02r01.{date_str}.nc")

        # Skip existing valid files
        if os.path.exists(output_file) and os.path.getsize(output_file) > MIN_VALID_SIZE:
            print(f"✅ Already exists: {output_file}")
            continue

        # Check if file exists on NOAA server
        response = requests.head(file_url)
        if response.status_code == 404:
            print(f"❌ File not found: {file_url}")
            continue

        # Download the file using requests
        print(f"🔽 Downloading {file_url} ...")
        print(f"Output file: {output_file}")  # Debugging output

        response = requests.get(file_url, stream=True)
        if response.status_code == 200:
            with open(output_file, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            # Validate download size
            if os.path.getsize(output_file) < MIN_VALID_SIZE:
                print(f"❌ File too small, deleting: {output_file}")
                os.remove(output_file)
                continue

            print(f"✅ Downloaded successfully: {output_file}")
        else:
            print(f"❌ Download failed: {file_url} (Status Code: {response.status_code})")


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor

BASE_URL = "https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/"
RAW_DIR = "downloaded_files"
MIN_VALID_SIZE = 1024
MAX_THREADS = 12  # Adjust this to control concurrency

os.makedirs(RAW_DIR, exist_ok=True)

def get_links(url, file_extension=None, is_directory=False):
    """Fetch all links from a NOAA directory webpage."""
    response = requests.get(url)
    if response.status_code != 200:
        print(f"❌ Failed to fetch: {url}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    links = []

    for link in soup.find_all("a"):
        href = link.get("href")
        if not href or href.startswith(".."):  # Ignore parent directory link
            continue

        full_url = urljoin(url, href)

        if is_directory and href.endswith("/"):  # Subfolders
            links.append(full_url)
        elif file_extension and href.endswith(file_extension):  # Files
            links.append(full_url)

    return links

def download_file(file_url):
    """Download a single file if it doesn't already exist."""
    filename = os.path.basename(file_url)
    output_file = os.path.join(RAW_DIR, filename)

    if os.path.exists(output_file) and os.path.getsize(output_file) > MIN_VALID_SIZE:
        print(f"✅ Already exists: {output_file}")
        return

    print(f"🔽 Downloading {file_url} ...")
    response = requests.get(file_url, stream=True)

    if response.status_code == 200:
        with open(output_file, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        if os.path.getsize(output_file) < MIN_VALID_SIZE:
            print(f"❌ File too small, deleting: {output_file}")
            os.remove(output_file)
            return

        print(f"✅ Downloaded: {output_file}")
    else:
        print(f"❌ Download failed: {file_url} (Status Code: {response.status_code})")

# Step 1: Get all year-month subdirectories
subdirectories = get_links(BASE_URL, is_directory=True)

# Step 2: Get all .nc file links
all_file_links = []
for subdir in subdirectories:
    all_file_links.extend(get_links(subdir, file_extension=".nc"))

# Step 3: Download files in parallel using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(download_file, all_file_links)


In [ ]:
import xarray as xr
import os
import gc

subset_files = []
batch_size = 32  # Process in batches of 5

file_list = sorted([f for f in os.listdir(RAW_DIR) if f.endswith(".nc")])

for i in range(0, len(file_list), batch_size):
    batch = file_list[i:i+batch_size]
    print(f"\n🔹 Processing batch {i//batch_size + 1}/{len(file_list)//batch_size + 1}...")

    for file in batch:
        file_path = os.path.join(RAW_DIR, file)

        try:
            # Open dataset with chunking to reduce memory use
            ds = xr.open_dataset(file_path, engine="netcdf4", chunks={"time": 1})

            # Subset region (California Coast)
            ds_subset = ds.sel(lat=LAT_RANGE, lon=LON_RANGE)

            # Keep only SST variable
            ds_subset = ds_subset[["sst"]]

            # Save subset file
            subset_file_path = os.path.join(SUBSET_DIR, file)
            ds_subset.to_netcdf(subset_file_path)

            # Close dataset to free memory
            ds_subset.close()
            ds.close()

            subset_files.append(subset_file_path)
            print(f"✅ Processed: {file}")

        except Exception as e:
            print(f"❌ Skipping corrupt file: {file_path} | Error: {e}")

    # Manually clear memory after each batch
    gc.collect()

print(f"\n✅ All {len(subset_files)} files saved in {SUBSET_DIR}")


In [ ]:
import os
import xarray as xr

subset_output_dir = "/content/oisst_california_subset"
corrupt_files = []

# Check all NetCDF files
for file in sorted(os.listdir(subset_output_dir)):
    if file.endswith(".nc"):
        file_path = os.path.join(subset_output_dir, file)
        try:
            ds = xr.open_dataset(file_path, engine="netcdf4")
            ds.close()
        except Exception as e:
            print(f"❌ Corrupt file detected: {file_path} | Error: {e}")
            corrupt_files.append(file_path)

# Remove corrupted files
if corrupt_files:
    print(f"\n🚨 Removing {len(corrupt_files)} corrupt files...")
    for bad_file in corrupt_files:
        os.remove(bad_file)
    print("✅ Corrupt files deleted.")
else:
    print("✅ No corrupt files found.")


In [ ]:
import xarray as xr

# Pick two files to compare metadata
file1 = "./content/oisst_california_subset/oisst-avhrr-v02r01.20100501.nc"
file2 = "./content/oisst_california_subset/oisst-avhrr-v02r01.20100601.nc"

# Open files
ds1 = xr.open_dataset(file1, engine="netcdf4")
ds2 = xr.open_dataset(file2, engine="netcdf4")

# Compare dimensions
print("\n🔹 Dimensions in File 1:")
print(ds1.dims)
print("\n🔹 Dimensions in File 2:")
print(ds2.dims)

# Compare coordinate variables
print("\n🔹 Coordinates in File 1:")
print(ds1.coords)
print("\n🔹 Coordinates in File 2:")
print(ds2.coords)

# Compare attributes
print("\n🔹 Global attributes in File 1:")
print(ds1.attrs)
print("\n🔹 Global attributes in File 2:")
print(ds2.attrs)

# Close datasets
ds1.close()
ds2.close()


In [ ]:
import xarray as xr
import os

subset_output_dir = "./content/oisst_california_subset"
nc_files = sorted([
    os.path.join(subset_output_dir, f) for f in os.listdir(subset_output_dir) if f.endswith(".nc")
])

# Check time variable for each file
for file in nc_files[:10]:  # Check first 10 files
    ds = xr.open_dataset(file, engine="netcdf4")
    print(f"\n🔹 {file}")
    print(ds.time)
    ds.close()


In [ ]:
import xarray as xr

# Select a few files for testing
test_files = [
    "./content/oisst_california_subset/oisst-avhrr-v02r01.20100501.nc",
    "./content/oisst_california_subset/oisst-avhrr-v02r01.20100601.nc",
    "./content/oisst_california_subset/oisst-avhrr-v02r01.20100701.nc",
    "./content/oisst_california_subset/oisst-avhrr-v02r01.20100801.nc",
    "./content/oisst_california_subset/oisst-avhrr-v02r01.20100901.nc"
]

try:
    ds_list = [xr.open_dataset(f, engine="netcdf4") for f in test_files]
    ds_combined = xr.concat(ds_list, dim="time")  # Merge along time axis

    print("✅ Test merge successful!")

    # Close datasets
    for ds in ds_list:
        ds.close()
except Exception as e:
    print(f"❌ Merge failed! Error: {e}")


In [ ]:
!apt-get install -y nco


In [ ]:
import subprocess

# Run the 'pwd' command to print the current working directory of the subprocess
result = subprocess.run('cd /mnt/c/ & pwd', shell=True, capture_output=True, text=True)

# Print the result
print("Subprocess working directory:", result.stdout)


In [ ]:
import os
import pyperclip
import subprocess

# Run the 'pwd' command to print the current working directory of the subprocess
result = subprocess.run(
    "pwd",
    shell=True,  # Use shell to execute the command
    check=True,  # Raise an error if the command fails
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Print the output
print(f"Subprocess working directory: {result.stdout.decode().strip()}")

subset_output_dir = r"./content/oisst_california_subset/"
final_output_file = r"./content/oisst_california_1981_2025.nc"

# Get sorted list of valid NetCDF files
valid_files = sorted([
    os.path.join(subset_output_dir, f) for f in os.listdir(subset_output_dir) if f.endswith(".nc")
])

# Define the batch size (adjust as needed)
batch_size = 100  # Process 100 files at a time, or adjust to your system's capacity
num_batches = (len(valid_files) // batch_size) + 1

# Intermediate output files for batch merging
temp_output_file = r"./content/temp_batch_merged.nc"

# Process files in batches
for i in range(num_batches):
    # Create a batch of files for the current batch
    batch_files = valid_files[i * batch_size: (i + 1) * batch_size]

    # Generate merge command for the batch
    merge_command = ["ncrcat"] + batch_files + [temp_output_file]

    # Copy the command to clipboard
    pyperclip.copy(" ".join(merge_command))

    print(f"\n🔹 Merging batch {i + 1}/{num_batches} of {len(valid_files)} files...")
    print(f"✔️ Merge command copied to clipboard for batch {i + 1}: \n{' '.join(merge_command)}")

    try:
        # Execute the batch merge command using subprocess
        result = subprocess.run(merge_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"✅ Batch {i + 1} merged successfully.")

    except subprocess.CalledProcessError as e:
        # Handle error if the command fails
        print(f"❌ Error: Command failed for batch {i + 1}")
        print(f"Error message: {e.stderr.decode()}")
        break

    # After each batch, append to the final output (or merge the batch incrementally)
    if i == 0:
        # For the first batch, use temp output file as the final result
        os.rename(temp_output_file, final_output_file)
    else:
        # For subsequent batches, append the merged result to the final output
        append_command = ["ncrcat", final_output_file, temp_output_file, final_output_file]

        try:
            # Execute the append command using subprocess
            result = subprocess.run(append_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(f"✅ Batch {i + 1} appended to final dataset.")
        except subprocess.CalledProcessError as e:
            print(f"❌ Error: Command failed while appending batch {i + 1} to the final output.")
            print(f"Error message: {e.stderr.decode()}")
            break

print(f"✅ Final dataset saved as {final_output_file}")


In [ ]:
import os

# Define the directories and files
subset_output_dir = "C:\\Users\\Brandt\\Documents\\UniversityOfChicago\\Courses\\31006_TimeSeriesAnalysis\\project\\data\\content\\oisst_california_subset\\"
final_output_file = "C:\\Users\\Brandt\\Documents\\UniversityOfChicago\\Courses\\31006_TimeSeriesAnalysis\\project\\data\\content\\oisst_california_1981_2025.nc"

# Get sorted list of valid NetCDF files
valid_files = sorted([
    os.path.join(subset_output_dir, f) for f in os.listdir(subset_output_dir) if f.endswith(".nc")
])

# Define the batch size (adjust as needed)
batch_size = 100  # Process 100 files at a time, or adjust to your system's capacity
num_batches = (len(valid_files) // batch_size) + 1

# Temporary output file for batch merging
temp_output_file = os.path.join(subset_output_dir, "temp_batch_merged.nc")

# Process files in batches
for i in range(num_batches):
    # Create a batch of files for the current batch
    batch_files = valid_files[i * batch_size: (i + 1) * batch_size]

    # Generate merge command for the batch
    merge_command = f"ncrcat {' '.join(batch_files)} {temp_output_file}"

    print(f"\n🔹 Merging batch {i + 1}/{num_batches} of {len(valid_files)} files...")
    print(f"✔️ Merge command copied to clipboard for batch {i + 1}: \n{merge_command}")

    # Execute the batch merge command
    result = os.system(merge_command)
    if result != 0:
        print(f"❌ Error: Command failed for batch {i + 1}")
        break
    else:
        print(f"✅ Batch {i + 1} merged successfully.")

    # After each batch, append to the final output (or merge the batch incrementally)
    if i == 0:
        # For the first batch, use temp output file as the final result
        os.rename(temp_output_file, final_output_file)
    else:
        # For subsequent batches, append the merged result to the final output
        append_command = f"ncrcat {final_output_file} {temp_output_file} {final_output_file}"
        result = os.system(append_command)
        if result != 0:
            print(f"❌ Error: Command failed while appending batch {i + 1} to the final output.")
            break
        else:
            print(f"✅ Batch {i + 1} appended to final dataset.")

# Clean up temporary files
if os.path.exists(temp_output_file):
    os.remove(temp_output_file)

print(f"✅ Final dataset saved as {final_output_file}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Move the final dataset
!mv /content/oisst_california_1981_2025.nc /content/drive/MyDrive/oisst_california_1981_2025.nc
